# Drug-Target Interaction Prediction using DeepDTA

| [Open In Colab](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/bindingdb_deepdta/tutorial.ipynb) (click `Runtime` → `Run all (Ctrl+F9)` |

구글 코랩 사용 시[Google Colab](https://colab.research.google.com), GPU를 사용해야 빠르게 동작시킬 수 있으니 상단의 메뉴바에서 `런타임` → `런타임 유형 변경` → `하드웨어 가속기: GPU` 로 꼭 변경할 것

## 개요
약물-표적 상호작용 예측은 약물 발견 분야에서 중요한 연구 분야이다. 주어진 화합물과 단백질 표적 사이의 결합 친화도를 예측하는 것을 말한다. 이 예에서 우리는 측정된 결합 친화도의 웹 액세스 가능 공개 데이터 세트인 BindingDB의 기준선으로 표준 DeepDTA 모델을 훈련힌다.

### DeepDTA
[DeepDTA](https://academic.oup.com/bioinformatics/article/34/17/i821/5093245) 는 합성곱 신경망(CNN)을 사용하여 단백질 시퀀스 및 복합 1D 표현을 모델링한다.

DeepDTA의 전체 아키텍처는 다음과 같습니다.

![DeepDTA](https://github.com/pykale/pykale/blob/main/examples/bindingdb_deepdta/figures/deepdta.png?raw=1)

### 데이터셋
서로 다른 Affinity 측정 지표로 구분되는 BindingDB에서 **3개의 데이터 세트**를 가져온다. (**Kd, IC50 및 Ki**).

이는 [Therapeutics Data Commons](https://tdcommons.ai/)(TDC)에서 가져왔으며, 이는 다양한 치료 영역에 걸쳐 퍼져 있는 기계 학습 작업이 모아져있는 사이트다. 데이터 통계는 다음과 같다.

|  Metrics   | Drugs | Targets | Pairs |
|  :----:  | :----:  |   :----:  | :----:  |
| Kd  | 10,655 | 1,413 | 52,284 |
| IC50  | 549,205 | 5,078 | 991,486 |
| Ki | 174,662 | 3,070 | 375,032 |

이 그림은 각각 메트릭 값(x축)이 로그 공간으로 변환된 3개의 데이터 세트에 대한 결합 친화도 분포다.
![Binding affinity distribution](https://github.com/pykale/pykale/blob/main/examples/bindingdb_deepdta/figures/bindingdb.jpg?raw=1)
이 노트북에서는 **Kd** 데이터셋을 사용한다.

## 설치

노트북 실행 환경을 설정하고 PyKale을 포함한 필수 모듈을 가져오려면 처음 몇 개의 코드 블록이 필요하다.

이렇게 하면 노트북이 Google Colab에서 실행 중인지 확인하고 필수 패키지를 설치한다.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip uninstall --yes imgaug && pip uninstall --yes albumentations && pip install git+https://github.com/aleju/imgaug.git
    !pip install git+https://github.com/pykale/pykale.git
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
    !pip install torch-geometric
    !pip install rdkit-pypi
    !git clone https://github.com/pykale/pykale.git
    %cd pykale/examples/bindingdb_deepdta/
else:
    print('Not running on CoLab')

Running on CoLab
Found existing installation: imgaug 0.2.9
Uninstalling imgaug-0.2.9:
  Successfully uninstalled imgaug-0.2.9
Found existing installation: albumentations 0.1.12
Uninstalling albumentations-0.1.12:
  Successfully uninstalled albumentations-0.1.12
  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-ear8bl6h
  Running command git clone -q https://github.com/aleju/imgaug.git /tmp/pip-req-build-ear8bl6h
  Created wheel for imgaug: filename=imgaug-0.4.0-py3-none-any.whl size=971122 sha256=9bac0d3e9aeae72f97c8712f5bdd0af11c0015d4e0f99f7dd3be3866ed718216
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjsc6eya/wheels/0c/78/b5/9303fae9d5e03df1f319adfe4e6534180b5c3232de11bc9a2f
Successfully built imgaug
  Cloning https://github.com/pykale/pykale.git to /tmp/pip-req-build-q12ukom2
  Running command git clone -q https://github.com/pykale/pykale.git /tmp/pip-req-build-q12ukom2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
 

This imports required modules.

In [ ]:
import pytorch_lightning as pl
import torch
from config import get_cfg_defaults
from model import get_model
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader, Subset

from kale.loaddata.tdc_datasets import BindingDBDataset
from kale.utils.seed import set_seed

In [ ]:
!pwd

/content/pykale/examples/bindingdb_deepdta


## 구성 정의

이 노트북에서 사용된 사용자 정의 구성은 `./configs/tutorial.yaml`에 저장되며, 이 파일은 값이 지정된 `config.py`의 기본값을 덮어쓴다.

이 노트북에서 전체 파이프라인을 실행하는 시간을 절약하기 위해 원본 BindingDB 데이터 세트에서 작은 훈련/유효한/테스트(8,000/1,000/1,000) 하위 집합을 샘플링할 것이다.

In [ ]:
cfg_path = "./configs/tutorial.yaml"
train_subset_size, valid_subset_size, test_subset_size = 8000, 1000, 1000

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

set_seed(cfg.SOLVER.SEED)

DATASET:
  NAME: BindingDB_Kd
  PATH: ./data
  Y_LOG: True
MODEL:
  DRUG_DIM: 128
  DRUG_FILTER_LENGTH: 8
  DRUG_LENGTH: 85
  MLP_DROPOUT_RATE: 0.2
  MLP_HIDDEN_DIM: 1024
  MLP_IN_DIM: 192
  MLP_OUT_DIM: 512
  NUM_ATOM_CHAR: 25
  NUM_FILTERS: 32
  NUM_SMILE_CHAR: 64
  TARGET_DIM: 128
  TARGET_FILTER_LENGTH: 8
  TARGET_LENGTH: 1200
SOLVER:
  LR: 0.001
  MAX_EPOCHS: 5
  MIN_EPOCHS: 0
  SEED: 2020
  TEST_BATCH_SIZE: 256
  TRAIN_BATCH_SIZE: 256


## GPU 사용할 수 있는지 확인하기

CUDA GPU를 사용할 수 있는 경우 이를 사용하여 교육 프로세스를 가속화해야 한다. 아래 코드는 이를 확인하고 보고한다.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: " + device)
gpus = 1 if device == "cuda" else 0

Using: cuda


## Dataset 선택

소스 및 대상 데이터 세트는 'BindingDBDataset()' 함수를 사용하여 지정하고 'DataLoader()' 함수를 사용하여 불러온다.

In [ ]:
train_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="train", path=cfg.DATASET.PATH)
valid_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="valid", path=cfg.DATASET.PATH)
test_dataset = BindingDBDataset(name=cfg.DATASET.NAME, split="test", path=cfg.DATASET.PATH)
train_size, valid_size, test_size = len(train_dataset), len(valid_dataset), len(test_dataset)
train_sample_indices, valid_sample_indices, test_sample_indices = torch.randperm(train_size)[:train_subset_size].tolist(), torch.randperm(valid_size)[:valid_subset_size].tolist(), torch.randperm(test_size)[:test_subset_size].tolist()
train_dataset, valid_dataset, test_dataset = Subset(train_dataset, train_sample_indices), Subset(valid_dataset, valid_sample_indices), Subset(test_dataset, test_sample_indices)

Downloading...
100%|██████████| 54.4M/54.4M [00:02<00:00, 27.0MiB/s]
Loading...
Done!
To log space...
Found local copy...
Loading...
Done!
To log space...
Found local copy...
Loading...
Done!
To log space...


In [ ]:
cfg.DATASET.PATH

'./data'

In [ ]:
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=cfg.SOLVER.TRAIN_BATCH_SIZE)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=True, batch_size=cfg.SOLVER.TEST_BATCH_SIZE)
test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=cfg.SOLVER.TEST_BATCH_SIZE)

## 모델 설정

여기에서 이전에 정의한 구성과 데이터 세트를 사용하여 이후에 학습할 모델을 설정한다.

In [ ]:
model = get_model(cfg)

In [ ]:
model

DeepDTATrainer(
  (drug_encoder): CNNEncoder(
    (embedding): Embedding(65, 128)
    (conv1): Conv1d(85, 32, kernel_size=(8,), stride=(1,))
    (conv2): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
    (conv3): Conv1d(64, 96, kernel_size=(8,), stride=(1,))
    (global_max_pool): AdaptiveMaxPool1d(output_size=1)
  )
  (target_encoder): CNNEncoder(
    (embedding): Embedding(26, 128)
    (conv1): Conv1d(1200, 32, kernel_size=(8,), stride=(1,))
    (conv2): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
    (conv3): Conv1d(64, 96, kernel_size=(8,), stride=(1,))
    (global_max_pool): AdaptiveMaxPool1d(output_size=1)
  )
  (decoder): MLPDecoder(
    (fc1): Linear(in_features=192, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=1024, bias=True)
    (fc3): Linear(in_features=1024, out_features=512, bias=True)
    (fc4): Linear(in_features=512, out_features=1, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

## 로거 설정

로거는 모델 훈련 중 및 후에 생성된 출력을 저장하는 데 사용된다. 이 정보는 교육의 효과를 평가하고 문제를 식별하는 데 사용할 수 있다.

In [ ]:
tb_logger = TensorBoardLogger("tb_logs", name=cfg.DATASET.NAME)

## 학습기 설정

트레이너 개체는 모델 매개변수를 결정하고 저장하는 데 사용됩니다. 여기서 하나는 모델을 훈련시키는 방법과 사용할 하드웨어에 대한 정보로 구성됩니다.

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="valid_loss", mode="min")
trainer = pl.Trainer(min_epochs=cfg.SOLVER.MIN_EPOCHS, 
                     max_epochs=cfg.SOLVER.MAX_EPOCHS, 
                     gpus=gpus, logger=tb_logger, 
                     callbacks=[checkpoint_callback])

## 모델 학습

학습기를 사용해서 모델 파라미터 최적화한다.

In [ ]:
%time trainer.fit(model, train_loader, valid_loader)

## 최적화된 모델 테스트

훈련에 사용되지 않은 테스트 데이터와 훈련 데이터로 최적화된 모델의 성능을 확인한다.

In [ ]:
trainer.test(model, test_loader)

RMSE(제곱 평균 제곱근)에서 대략 $7.3\cdots$의 테스트 손실을 얻었을 것이다. 타겟 값($y$)의 범위는 [-13, 20](로그 공간)인 것을 앞선 그래프로 알 수 있을 것이다. 우리는 단 3개의 Epoch로 [-13, 20] 범위에서 대략 7.3의 RMSE로 목표 값을 예측할 수 있다는 것을 확인할 수 있다.

이 노트북을 실행하는 시간을 절약하기 위해 최대 epoch를 3으로 설정하고 하위 집합(8000/1000/1000)을 추출한다. 이러한 설정은 사용자가 변경할 수 있다. 최대 에포크를 100으로 설정하고 전체 데이터 세트를 사용하면 훨씬 더 나은 결과를 얻을 수 있을 것이다(<1.0 이하의 loss).

## 건축물
다음은 기본 하이퍼파라미터 설정이 있는 DeepDTA의 아키텍처다.

<pre>
==========================================================================================
Layer (type:depth-idx)                   Output Shape              Param #
==========================================================================================
├─CNNEncoder: 1-1                        [256, 96]                 --
|    └─Embedding: 2-1                    [256, 85, 128]            8,320
|    └─Conv1d: 2-2                       [256, 32, 121]            21,792
|    └─Conv1d: 2-3                       [256, 64, 114]            16,448
|    └─Conv1d: 2-4                       [256, 96, 107]            49,248
|    └─AdaptiveMaxPool1d: 2-5            [256, 96, 1]              --
├─CNNEncoder: 1-2                        [256, 96]                 --
|    └─Embedding: 2-6                    [256, 1200, 128]          3,328
|    └─Conv1d: 2-7                       [256, 32, 121]            307,232
|    └─Conv1d: 2-8                       [256, 64, 114]            16,448
|    └─Conv1d: 2-9                       [256, 96, 107]            49,248
|    └─AdaptiveMaxPool1d: 2-10           [256, 96, 1]              --
├─MLPDecoder: 1-3                        [256, 1]                  --
|    └─Linear: 2-11                      [256, 1024]               197,632
|    └─Dropout: 2-12                     [256, 1024]               --
|    └─Linear: 2-13                      [256, 1024]               1,049,600
|    └─Dropout: 2-14                     [256, 1024]               --
|    └─Linear: 2-15                      [256, 512]                524,800
|    └─Linear: 2-16                      [256, 1]                  513
==========================================================================================
Total params: 2,244,609
Trainable params: 2,244,609
Non-trainable params: 0
Total mult-adds (M): 58.08
==========================================================================================
Input size (MB): 1.32
Forward/backward pass size (MB): 429.92
Params size (MB): 8.98
Estimated Total Size (MB): 440.21